In [1]:
import numpy as np
import pandas as pd
import os
import string
import nltk
import math
from textblob import TextBlob as tb

from nltk.corpus import brown
from nltk.corpus import reuters

from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer

from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer
from nltk.stem import SnowballStemmer

In [2]:
data = pd.read_pickle("./data/data_words.pkl")

In [ ]:
data = pd.read_csv("data/articles1.csv") 

In [3]:
data

,Unnamed: 0,id,title,publication,author,date,year,month,url,content,words
0,0,17283,House Republicans Fret About Winning Their Hea...,New York Times,Carl Hulse,2016-12-31,2016.0,12.0,NaN,WASHINGTON — Congressional Republicans have...,"[washington, congression, republican, new, fea..."
1,1,17284,Rift Between Officers and Residents as Killing...,New York Times,Benjamin Mueller and Al Baker,2017-06-19,2017.0,6.0,NaN,"After the bullet shells get counted, the blood...","[bullet, shell, get, count, blood, dri, votiv,..."
2,2,17285,"Tyrus Wong, ‘Bambi’ Artist Thwarted by Racial ...",New York Times,Margalit Fox,2017-01-06,2017.0,1.0,NaN,"When Walt Disney’s “Bambi” opened in 1942, cri...","[walt, disney, ’, “, bambi, ”, open, 1942, cri..."
3,3,17286,"Among Deaths in 2016, a Heavy Toll in Pop Musi...",New York Times,William McDonald,2017-04-10,2017.0,4.0,NaN,"Death may be the great equalizer, but it isn’t...","[death, may, great, equal, ’, necessarili, eve..."
4,4,17287,Kim Jong-un Says North Korea Is Preparing to T...,New York Times,Choe Sang-Hun,2017-01-02,2017.0,1.0,NaN,"SEOUL, South Korea — North Korea’s leader, ...","[seoul, south, korea, north, korea, ’, leader,..."
5,5,17288,"Sick With a Cold, Queen Elizabeth Misses New Y...",New York Times,Sewell Chan,2017-01-02,2017.0,1.0,NaN,"LONDON — Queen Elizabeth II, who has been b...","[london, queen, elizabeth, ii, battl, cold, we..."
6,6,17289,Taiwan’s President Accuses China of Renewed In...,New York Times,Javier C. Hernández,2017-01-02,2017.0,1.0,NaN,BEIJING — President Tsai of Taiwan sharpl...,"[beij, presid, tsai, taiwan, sharpli, critic, ..."
7,7,17290,"After ‘The Biggest Loser,’ Their Bodies Fought...",New York Times,Gina Kolata,2017-02-08,2017.0,2.0,NaN,"Danny Cahill stood, slightly dazed, in a blizz...","[danni, cahil, stood, slight, daze, blizzard, ..."
8,8,17291,"First, a Mixtape. Then a Romance. - The New Yo...",New York Times,Katherine Rosman,2016-12-31,2016.0,12.0,NaN,"Just how is Hillary Kerr, the founder of ...","[hillari, kerr, founder, digit, media, compani..."
9,9,17292,Calling on Angels While Enduring the Trials of...,New York Times,Andy Newman,2016-12-31,2016.0,12.0,NaN,Angels are everywhere in the Muñiz family’s ap...,"[angel, everywher, muñiz, famili, ’, apart, br..."


In [4]:
for s in data['content'][0:2]:
    print(s + '\n~\n')

WASHINGTON  —   Congressional Republicans have a new fear when it comes to their    health care lawsuit against the Obama administration: They might win. The incoming Trump administration could choose to no longer defend the executive branch against the suit, which challenges the administration’s authority to spend billions of dollars on health insurance subsidies for   and   Americans, handing House Republicans a big victory on    issues. But a sudden loss of the disputed subsidies could conceivably cause the health care program to implode, leaving millions of people without access to health insurance before Republicans have prepared a replacement. That could lead to chaos in the insurance market and spur a political backlash just as Republicans gain full control of the government. To stave off that outcome, Republicans could find themselves in the awkward position of appropriating huge sums to temporarily prop up the Obama health care law, angering conservative voters who have been d

In [ ]:
exclude = string.punctuation + '—'

stop_words = stopwords.words('english')

snowball_stemmer = SnowballStemmer("english")
temp = []
for i,s in enumerate(data['content']):
    s = s.translate(str.maketrans('', '', exclude))
    s = word_tokenize(s)
    s = [x.lower() for x in s]
    s = [w for w in s if not w in stop_words]
    s = [snowball_stemmer.stem(w) for w in s]
    print(i)
    #print(str(s) + '\n~\n')
    temp.append(s)
data['words'] = temp

In [ ]:
data.to_pickle("./data/data_words.pkl")

In [7]:
unique_words = list(set([item for sublist in data['words'].values.flatten() for item in sublist]))

In [5]:
def tf(word, doc):
    return doc.count(word) / len(doc)

def n_containing(word, doclist):
    return sum(1 for doc in doclist if word in doc)

def idf(word, doclist):
    return math.log(len(doclist) / (0.01 + n_containing(word, doclist)))

def tfidf(word, doc, doclist):
    return (tf(word, doc) * idf(word, doclist))

In [ ]:
worddic = {}

d = data['words']

for i,doc_words in enumerate(d):
    print(i)
    for word in unique_words:
        if word in doc_words:
            positions = list(np.where(np.array(doc_words) == word)[0])
            idfs = tfidf(word,doc_words,d)
            try:
                worddic[word].append([i,positions,idfs])
            except:
                worddic[word] = []
                worddic[word].append([i,positions,idfs])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84


In [15]:
np.save('data/worddic.npy', worddic)

[[0, [0], 0.00173725066858473],
 [1, [1572], 0.0003295325826453545],
 [6, [186, 206, 287], 0.007271569111476668],
 [7, [371], 0.0005122162845715067]]

In [185]:
from collections import Counter

def search(searchsentence):
    try:
            # split sentence into individual words 
        searchsentence = searchsentence.lower()
        try:
            words = searchsentence.split(' ')
        except:
            words = list(words) #why???
        enddic = {}
        idfdic = {}
        closedic = {}

        # remove words if not in worddic 
        words = [x for x in words if x in list(worddic.keys())]
        numwords = len(words)

        # make metric of number of occurances of all words in each doc & largest total IDF 
        for word in words:
            for indpos in worddic[word]:
                index = indpos[0]
                amount = len(indpos[1])
                idfscore = indpos[2]
                enddic[index] = amount
                idfdic[index] = idfscore
                fullcount_order = sorted(enddic.items(), key=lambda x:x[1], reverse=True)
                fullidf_order = sorted(idfdic.items(), key=lambda x:x[1], reverse=True)


        # make metric of what percentage of words appear in each doc
        combo = []
        alloptions = {k: worddic.get(k, None) for k in (words)}
        for worddex in list(alloptions.values()):
            for indexpos in worddex:
                for indexz in indexpos:
                    combo.append(indexz)
        comboindex = combo[::3]
        combocount = Counter(comboindex)
        for key in combocount:
            combocount[key] = combocount[key] / numwords
        combocount_order = sorted(combocount.items(), key=lambda x:x[1], reverse=True)

        # make metric for if words appear in same order as in search
        if len(words) > 1:
            x = []
            y = []
            for record in [worddic[z] for z in words]:
                for index in record:
                     x.append(index[0])
            for i in x:
                if x.count(i) > 1:
                    y.append(i)
            y = list(set(y)) #articles where > 2 search words appear

            closedic = {}
            for wordbig in [worddic[x] for x in words]:
                for record in wordbig:
                    if record[0] in y:
                        index = record[0]
                        positions = record[1]
                        try:
                            closedic[index].append(positions)
                        except:
                            closedic[index] = []
                            closedic[index].append(positions) #dict with articles where >2 search words appear, doc: [word positions]

            x = 0
            fdic = {}
            for index in y:
                csum = []
                for seqlist in closedic[index]:
                    while x > 0:
                        secondlist = seqlist
                        x = 0
                        #sol = [1 for i in firstlist if i + 1 in secondlist] #if position+1 is also in list
                        
                        sol=[]
                        for i in firstlist:
                            for aux in range(1,5):
                                if i + aux in secondlist:
                                    sol.append(1/aux)
                                    
                        csum.append(sol)
                        fsum = [item for sublist in csum for item in sublist]
                        fsum = sum(fsum)
                        fdic[index] = fsum
                        fdic_order = sorted(fdic.items(), key=lambda x:x[1], reverse=True) #higher score for consecutive words
                    while x == 0:
                        firstlist = seqlist
                        x = x + 1
        else:
            fdic_order = [(-1,-1)]

        # also the one above should be given a big boost if ALL found together 


        #could make another metric for if they are not next to each other but still close 


        return(searchsentence,words,fullcount_order,combocount_order,fullidf_order,fdic_order)

    except:
        return(None)


In [186]:
search('trump')

('trump',
 ['trump'],
 [(0, 9), (4, 5), (6, 5)],
 [(0, 1.0), (4, 1.0), (6, 1.0)],
 [(0, 0.020543355756842876),
  (6, 0.015923673928823098),
  (4, 0.013310920335180283)],
 [(-1, -1)])

In [196]:
def rank(term):
    results = search(term)

    if results:
        # get metrics 
        num_score = results[2]
        per_score = results[3]
        tfscore = results[4]
        order_score = results[5]

        final_candidates = []

        # rule1: if high word order score & 100% percentage terms then put at top position
        try:
            first_candidates = []

            for candidates in order_score:
                if candidates[1] > 1:
                    first_candidates.append(candidates[0])

            second_candidates = []

            for match_candidates in per_score:
                if match_candidates[1] == 1:
                    second_candidates.append(match_candidates[0])
                if match_candidates[1] == 1 and match_candidates[0] in first_candidates:
                    final_candidates.append(match_candidates[0])

        # rule2: next add other word order score which are greater than 1 

            t3_order = first_candidates[0:3]
            for each in t3_order:
                if each not in final_candidates:
                    final_candidates.insert(len(final_candidates),each)

        # rule3: next add top td-idf results
            final_candidates.insert(len(final_candidates),tfscore[0][0])
            final_candidates.insert(len(final_candidates),tfscore[1][0])

        # rule4: next add other high percentage score 
            t3_per = second_candidates[0:3]
            for each in t3_per:
                if each not in final_candidates:
                    final_candidates.insert(len(final_candidates),each)

        #rule5: next add any other top results for metrics
            othertops = [num_score[0][0],per_score[0][0],tfscore[0][0],order_score[0][0]]
            for top in othertops:
                if top not in final_candidates:
                    if top > -1:
                        final_candidates.insert(len(final_candidates),top)

        # unless single term searched, in which case just return 
        except:
            othertops = [num_score[0][0],num_score[1][0],num_score[2][0],per_score[0][0],tfscore[0][0]]
            for top in othertops:
                if top not in final_candidates:
                    final_candidates.insert(len(final_candidates),top)

        final_candidates_score = []
        for temp in final_candidates[0:20]:
            a = sum([item[1] for item in num_score if item[0] == temp]+
                    [item[1] for item in per_score if item[0] == temp]+
                    [item[1] for item in tfscore if item[0] == temp]+
                    [item[1] for item in order_score if item[0] == temp])
            final_candidates_score.append(a)
            
        return(len(final_candidates),final_candidates[0:20], final_candidates_score)
    else:
        return(0,[],[])

In [197]:
rank('trump')

(3, [0, 6, 4], [10.020543355756843, 6.015923673928823, 6.01331092033518])